In [2]:
from datasets import load_dataset, IterableDataset

In [52]:
import json
from brx import BRX, sftoq, uif
from dotenv import load_dotenv
import os
from openai import OpenAI
from pydantic import BaseModel, Field

# load_dotenv("../../.env")
brx_client = BRX(os.environ["BRX_ACCESS_TOKEN"])
openai_client = OpenAI(api_key=os.environ["TOGETHER_API_KEY"], base_url="https://api.together.xyz")

class GeneratedBreakdown(BaseModel):
    one_sentence_instruction: str = Field(description="Rewrite the original commit message to be a succinct instruction on what to code. The instruction should be one sentence and be brief.")
    plan_no_code: str = Field(description="Write a numbered step-by-step plan that would be followed to generate the patch. State variable and function names, but do not write lines of code under any circumstance.")
    plan_with_code: str = Field(description="Write a numbered step-by-step plan that would be followed to generate the patch. Write a short snippet of code after the instruction text if useful. You must use instructional text as well as code.")

def call_openai(file_contents, original_commit_message, applied_patch):
    system_prompt = "Given an original file, a starting commit message, and a patch applied, write a one-sentence description of changes made and two numbered step-by-step plans for generating those changes: one that includes code and one that doesn't."
    user_prompt = f"# File contents: \n {file_contents} \n \n ----- \n \n# Original commit message: \n{original_commit_message}\n\n -----\n# Applied Patch\n{applied_patch}"
    response = openai_client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        response_format={"type": "json_object", "schema": GeneratedBreakdown.model_json_schema()}
    )
    generated_breakdown = json.loads(response.choices[0].message.content)
    return generated_breakdown

def call_brx(file_contents, original_commit_message, applied_patch):
    schema = json.dumps({"description":"Given an original file, a starting commit message, and a patch applied, write a one-sentence description of changes made and a numbered step-by-step plan for generating those changes.","brxName":"Synthetic Dataset Generator for Patches","brxId":"bc7c33e6-8f01-4a34-8846-e084eaae4e84","dependantBrxIds":{},"processType":8,"schemas":{"mainBrxId":"bc7c33e6-8f01-4a34-8846-e084eaae4e84","schemas":{"_isMap":True,"data":[["main_brx_entry_schema",{"schemaFields":{"_isMap":True,"data":[["file_contents",{"fieldValueDataType":"string","fieldValue":"testval"}],["original_commit_message",{"fieldValueDataType":"string","fieldValue":"testval"}],["applied_patch",{"fieldValueDataType":"string","fieldValue":"testval"}]]},"brxName":"Synthetic Dataset Generator for Patches","brxId":"bc7c33e6-8f01-4a34-8846-e084eaae4e84"}]]}}})
    query_rebuild = sftoq(schema)
    output_object = query_rebuild["brx_query"]
    input_fields = query_rebuild["input_fields"]

    input_dict = {
        "file_contents": file_contents,
        "original_commit_message": original_commit_message,
        "applied_patch": applied_patch
    }

    for index, field in enumerate(input_fields):
        for dict_key in input_dict.keys():
            if field["name"] == dict_key:
                input_fields[index]["value"] = input_dict[dict_key]
        
    updated_query = uif(input_fields, output_object)
    result = json.loads(brx_client.execute(updated_query["brx_query"])["brxRes"]["output"])
    return result

In [55]:
import difflib
import re

def process_row_brx(row):
    # Input: original commit message, original file, generated patch
    # Output: rewritten description of changes made, plan generation
    commit_msg = row["subject"]
    original_file = row["old_contents"]
    udiff = "\n".join(difflib.unified_diff(row['old_contents'].splitlines(), row['new_contents'].splitlines(), fromfile=row['old_file'], tofile=row['new_file'], n=3))
    udiff = f"```diff\n{udiff}\n```"
    udiff = re.sub("@@.*@@", "@@...@@", udiff)

    generated_info = call_brx(commit_msg, original_file, udiff)
    return {"synthetic": True, "patch": udiff, "patch_description": generated_info["patch_description"], "plan_no_code": generated_info["plan_no_code"], "plan_with_code": generated_info["plan_with_code"]}

def process_row_together(row):
    udiff = "\n".join(difflib.unified_diff(row['old_contents'].splitlines(), row['new_contents'].splitlines(), fromfile=row['old_file'], tofile=row['new_file'], n=3))
    udiff = f"```diff\n{udiff}\n```"
    udiff = re.sub("@@.*@@", "@@...@@", udiff)
    try:
        print("Processing the row with Together")


        extracted_info = call_openai(row['old_contents'], udiff, row['subject'])
        extracted_info["synthetic"] = True
        extracted_info["patch"] = udiff 
        return extracted_info
    except:
        print("There was an error with this one")
        return {
            "synthetic": False,
            "patch": udiff,
            "one_sentence_instruction": "",
            "plan_no_code": "",
            "plan_with_code": ""
        }

def process_row(row):
    # Input: original commit message, original file, generated patch
    # Output: rewritten description of changes made, plan generation
    commit_msg = row["subject"]
    original_file = row["old_contents"]
    udiff = "\n".join(difflib.unified_diff(row['old_contents'].splitlines(), row['new_contents'].splitlines(), fromfile=row['old_file'], tofile=row['new_file'], n=3))
    udiff = f"```diff\n{udiff}\n```"
    udiff = re.sub("@@.*@@", "@@...@@", udiff)

    generated_info = call_brx(commit_msg, original_file, udiff)
    return {"synthetic": True, "patch": udiff, "patch_description": generated_info["patch_description"], "plan_no_code": generated_info["plan_no_code"], "plan_with_code": generated_info["plan_with_code"]}

In [43]:
languages = ["python", "jsx", "kotlin", "vue", "xml", "javascript", "java", "c#", "ruby", "c", "c++", "typescript", "dart"]
generated_datasets = []
for language in languages:
    dataset = load_dataset("bigcode/commitpackft", language, split="train")
    dataset = dataset.shuffle(seed=42).select(range(15))
    generated_datasets.append(dataset)

In [62]:
from datasets import concatenate_datasets, Dataset

ds = concatenate_datasets(generated_datasets)
ds = ds.map(lambda row: process_row_together(row), num_proc=16)

/Users/vijaydaita/miniconda3/envs/superdocs/lib/python3.12/site-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class '__main__.GeneratedBreakdown'>.
  StockPickler.save(self, obj, save_persistent_id)
/Users/vijaydaita/miniconda3/envs/superdocs/lib/python3.12/site-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class '__main__.GeneratedBreakdown'>: __main__.GeneratedBreakdown has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)
Map (num_proc=16):   0%|          | 0/195 [00:00<?, ? examples/s]

Processing the row with Together


Processing the row with TogetherProcessing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with TogetherProcessing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
Processing the row with Together
There was an error with this one


Map (num_proc=16):   1%|          | 1/195 [00:13<43:56, 13.59s/ examples]

Processing the row with Together
There was an error with this one


Map (num_proc=16):   1%|          | 2/195 [00:20<31:45,  9.87s/ examples]

...

Map (num_proc=16):  98%|█████████▊| 192/195 [23:13<00:21,  7.25s/ examples]

Processing the row with Together


Map (num_proc=16):  99%|█████████▉| 193/195 [23:17<00:12,  6.27s/ examples]

There was an error with this one


Map (num_proc=16): 100%|██████████| 195/195 [23:26<00:00,  7.21s/ examples]


In [63]:
ds.push_to_hub("vdaita/commitpackft-small-patches-expanded")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/vdaita/commitpackft-small-patches-expanded/commit/fc7c5076d0b65472c6c1c8608e015f6e83e789e0', commit_message='Upload dataset', commit_description='', oid='fc7c5076d0b65472c6c1c8608e015f6e83e789e0', pr_url=None, pr_revision=None, pr_num=None)